# All the parameters that need to be changed

In [1]:
# Hawaii
state_ab = "hi"

## Data
1. Download all the data in directory "il_data"
2. Eextract them all

In [36]:
data_folder = state_ab + "_data/"
population1_data = "./{}{}_pl2020_b/{}_pl2020_p1_b.shp".format(data_folder, state_ab, state_ab)
population2_data = "./{}{}_pl2020_b/{}_pl2020_p2_b.shp".format(data_folder, state_ab, state_ab)
vap_data =  "./{}{}_pl2020_b/{}_pl2020_p4_b.shp".format(data_folder, state_ab, state_ab)
vest20_data = "./{}{}_vest_20/{}_vest_20.shp".format(data_folder, state_ab, state_ab)
vest18_data = "./{}{}_vest_18/{}_vest_18.shp".format(data_folder, state_ab, state_ab)
vest16_data = "./{}{}_vest_16/{}_vest_16.shp".format(data_folder, state_ab, state_ab)
cd_data = "./{}{}_cong_adopted_2022/Congressional_Final_2022.shp".format(data_folder, state_ab)
send_data = "./{}{}_sldu_adopted_2022/hi_sldu_adopted_2022.shp".format(data_folder, state_ab)
hdist_data = "./{}{}_sldl_adopted_2022/hi_sldl_adopted_2022.shp".format(data_folder, state_ab)

## Parameters that needs to be manually checked

### base vest data
start_col = 5\
vest_base_data = vest20\
year = '20'

### district data
district column name of cong_df, send, hdist when calling add_dist()

# Program starts

In [3]:
import pandas as pd
import geopandas as gpd
import maup
import time
from maup import smart_repair
from gerrychain import Graph
import os

maup.progress.enabled = True

/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


In [4]:
import warnings
warnings.filterwarnings("ignore")

In [5]:
def do_smart_repair(df):
    # change it to the UTM it needs for smart_repair
    df = df.to_crs(df.estimate_utm_crs())
    df = smart_repair(df)
    
    # check maup doctor again to see if smart repair works
    if maup.doctor(df) == True:
        # change it back to this UTM for this data
        df = df.to_crs('EPSG:4269')
    else:
        raise Exception('maup.doctor failed')
    
    return df

In [6]:
def add_district(dist_df, dist_name, election_df, col_name):
    # check if it needs to be smart_repair
    if maup.doctor(dist_df) != True:
        dist_df = do_smart_repair(dist_df)

    election_df = gpd.GeoDataFrame(election_df, crs="EPSG:4269")

    # assigne the pricincts
    precincts_to_district_assignment = maup.assign(election_df.geometry, dist_df.geometry)
    election_df[dist_name] = precincts_to_district_assignment
    for precinct_index in range(len(election_df)):
        election_df.at[precinct_index, dist_name] = dist_df.at[election_df.at[precinct_index, dist_name], col_name]

    return election_df

In [7]:
def rename(original, year):
    party = original[6]
    if party == 'R' or party == 'D':
        return original[3:6] + year + original[6]
    else:
        return original[3:6] + year + 'O'

In [8]:
def check_population(population, df):
    pop_check = pd.DataFrame({
        'pop_col': pop_col,
        'population_df': population[pop_col].sum(), 
        'vest_base': df[pop_col].sum(),
        'equal': [x == y for x, y in zip(population[pop_col].sum(), df[pop_col].sum())]
    })
    if pop_check['equal'].mean() < 1:
        print(pop_check)
        raise Exception("population doesn't agree")

    else:
        print("population agrees")

In [9]:
def add_vest(vest, df, year, population, start_col):    
     # check if it needs to be smart_repair
    if maup.doctor(vest) != True:
        vest = do_smart_repair(vest)
    
    # rename the columns
    original_col = vest.columns[start_col:-1]
    new_col = [rename(i, year) for i in original_col]
    rename_dict = dict(zip(original_col, new_col))
    vest = vest.rename(columns=rename_dict)
    vest = vest.groupby(level=0, axis=1).sum() # combine all the other party's vote into columns with sufix "O"
    col_name = list(set(new_col))
    col_name.sort()
    
    # make the blocks from precincts by weight
    vest = gpd.GeoDataFrame(vest, crs="EPSG:4269")
    election_in_block = population[["VAP", 'geometry']] # population_df is in block scale
    blocks_to_precincts_assignment = maup.assign(election_in_block.geometry, vest.geometry)
    weights = election_in_block["VAP"] / blocks_to_precincts_assignment.map(election_in_block["VAP"].groupby(blocks_to_precincts_assignment).sum())
    weights = weights.fillna(0)
    prorated = maup.prorate(blocks_to_precincts_assignment, vest[col_name], weights)
    election_in_block[col_name] = prorated
    
    # assign blocks to precincts
    election_in_block = gpd.GeoDataFrame(election_in_block, crs="EPSG:4269")
    df = gpd.GeoDataFrame(df, crs="EPSG:4269")
    block_to_pricinct_assginment = maup.assign(election_in_block.geometry, df.geometry)
    df[col_name] = election_in_block[col_name].groupby(block_to_pricinct_assginment).sum()
    df = df.groupby(level=0, axis=1).sum()
    
    # check if population agrees
    check_population(population, df)
    
    return df

## Read the census data

In [10]:
population1_df = gpd.read_file(population1_data)
population2_df = gpd.read_file(population2_data)
vap_df = gpd.read_file(vap_data)

In [11]:
population2_df = population2_df.drop(columns=['SUMLEV', 'LOGRECNO', 'GEOID', 'COUNTY', 'geometry'])
vap_df = vap_df.drop(columns=['SUMLEV', 'LOGRECNO', 'GEOID', 'COUNTY', 'geometry'])

In [12]:
population_df = pd.merge(population1_df, population2_df, on='GEOID20')
population_df = pd.merge(population_df, vap_df, on='GEOID20')

In [13]:
rename_dict = {'P0020001': 'TOTPOP', 'P0020002': 'HISP', 'P0020005': 'NH_WHITE', 'P0020006': 'NH_BLACK', 'P0020007': 'NH_AMIN',
               'P0020008': 'NH_ASIAN', 'P0020009': 'NH_NHPI', 'P0020010': 'NH_OTHER', 'P0020011': 'NH_2MORE',
               'P0040001': 'VAP', 'P0040002': 'HVAP', 'P0040005': 'WVAP', 'P0040006': 'BVAP', 'P0040007': 'AMINVAP',
               'P0040008': 'ASIANVAP', 'P0040009': 'NHPIVAP', 'P0040010': 'OTHERVAP', 'P0040011': '2MOREVAP'}

In [14]:
population_df.rename(columns=rename_dict, inplace = True)

In [15]:
population_df['H_WHITE'] = population_df.apply(lambda t: t['P0010003'] - t['NH_WHITE'], 1)
population_df['H_BLACK'] = population_df.apply(lambda t: t['P0010004'] - t['NH_BLACK'], 1)
population_df['H_AMIN'] = population_df.apply(lambda t: t['P0010005'] - t['NH_AMIN'], 1)
population_df['H_ASIAN'] = population_df.apply(lambda t: t['P0010006'] - t['NH_ASIAN'], 1)
population_df['H_NHPI'] = population_df.apply(lambda t: t['P0010007'] - t['NH_NHPI'], 1)
population_df['H_OTHER'] = population_df.apply(lambda t: t['P0010008'] - t['NH_OTHER'], 1)
population_df['H_2MORE'] = population_df.apply(lambda t: t['P0010009'] - t['NH_2MORE'], 1)

# Read the base vest data
Now using it as a "base precinct", but it could be vest 18 or vest 16 if vest 20 is not working

In [16]:
def add_vest_base(vest, start_col, year):
    original_col = vest.columns[start_col:-1]
    new_col = [rename(i, year) for i in original_col]
    rename_dict = dict(zip(original_col, new_col))
    vest = vest.rename(columns=rename_dict)
    vest = vest.groupby(level=0, axis=1).sum()
    vest = gpd.GeoDataFrame(vest, crs="EPSG:4269")
    
    return vest

### Check if vest 20 can be used as base

In [17]:
vest20 = gpd.read_file(vest20_data)

In [18]:
if maup.doctor(vest20) != True:
    vest20 = do_smart_repair(vest20)

100%|██████████████████████████████████████████████| 262/262 [00:01<00:00, 185.39it/s]


There are 2 overlaps.
There are 4 holes.
Snapping all geometries to a grid with precision 10^( -5 ) to avoid GEOS errors.
Identifying overlaps...


100%|█████████████████████████████████████████████| 276/276 [00:00<00:00, 1277.09it/s]


Resolving overlaps...
Assigning order 2 pieces...
Filling gaps...


100%|██████████████████████████████████████████████| 262/262 [00:00<00:00, 263.06it/s]


### If it is true for maup doctor, we will use it as the base vest data.
Check where the election column starts, this should be the same for all vest data in that state

In [19]:
vest20.columns

Index(['COUNTYFP', 'STATEFP', 'PRECINCT', 'GEOID', 'G20PREDBID', 'G20PRERTRU',
       'G20PRELJOR', 'G20PREGHAW', 'G20PRECBLA', 'G20PREOPIE', 'geometry'],
      dtype='object')

## Parameters that need to be checked

In [20]:
start_col = 4
vest_base_data = vest20
year = '20'

In [21]:
vest_base = add_vest_base(vest_base_data, start_col, year)

In [22]:
# vap and population have the same GEOID20
blocks_to_precincts_assignment = maup.assign(population_df.geometry, vest_base.geometry)

100%|███████████████████████████████████████████████| 262/262 [00:07<00:00, 36.28it/s]


In [23]:
pop_col = ['TOTPOP', 'HISP', 'NH_WHITE', 'NH_BLACK', 'NH_AMIN', 'NH_ASIAN', 'NH_NHPI', 'NH_OTHER', 'NH_2MORE', 'H_WHITE', 'H_BLACK', 'H_AMIN', 'H_ASIAN', 'H_NHPI', 'H_OTHER', 'H_2MORE', 'VAP', 'HVAP', 'WVAP', 'BVAP', 'AMINVAP', 'ASIANVAP', 'NHPIVAP', 'OTHERVAP', '2MOREVAP']

In [24]:
vest_base[pop_col] = population_df[pop_col].groupby(blocks_to_precincts_assignment).sum()

In [25]:
election_df = gpd.GeoDataFrame(vest_base, crs="EPSG:4269")

### Check if the population agrees

In [26]:
check_population(population_df, vest_base)

population agrees


# Add more vest data

In [27]:
vest18 = gpd.read_file(vest18_data)

In [30]:
vest18.columns

Index(['COUNTY', 'PRECINCT', 'G18USSDHIR', 'G18USSRCUR', 'G18GOVDIGE',
       'G18GOVRTUP', 'G18GOVGBRE', 'G18GOVITER', 'geometry'],
      dtype='object')

In [31]:
# check the result here
start_col = 2
election_df = add_vest(vest18, election_df, '18', population_df, start_col)

100%|██████████████████████████████████████████████| 262/262 [00:00<00:00, 278.89it/s]


There are 2 overlaps.
There are 4 holes.
There are some invalid geometries.
Snapping all geometries to a grid with precision 10^( -5 ) to avoid GEOS errors.
Identifying overlaps...


100%|█████████████████████████████████████████████| 276/276 [00:00<00:00, 1605.60it/s]

Resolving overlaps...


Assigning order 2 pieces...
Filling gaps...


100%|███████████████████████████████████████████████| 262/262 [00:07<00:00, 35.53it/s]


population agrees


In [32]:
# vest 16

In [33]:
vest16 = gpd.read_file(vest16_data)
vest16.columns

Index(['precinct', 'G16PREDCli', 'G16PRERTru', 'G16PREGSte', 'G16PRELJoh',
       'G16PRECCas', 'G16USSDSch', 'G16USSRCar', 'G16USSAGiu', 'G16USSLKok',
       'G16USSCAll', 'geometry'],
      dtype='object')

In [34]:
start_col = 1
election_df = add_vest(vest16, election_df, '16', population_df, start_col)

100%|███████████████████████████████████████████████| 264/264 [00:04<00:00, 60.45it/s]


There are 12 overlaps.
There are 116 holes.
Snapping all geometries to a grid with precision 10^( -4 ) to avoid GEOS errors.
Identifying overlaps...


100%|██████████████████████████████████████████████| 551/551 [00:03<00:00, 160.69it/s]


Resolving overlaps...
Assigning order 2 pieces...
Filling gaps...


100%|███████████████████████████████████████████████| 262/262 [00:06<00:00, 38.01it/s]


population agrees


## Add the district data

In [43]:
cong_df = gpd.read_file(cd_data).to_crs('EPSG:4269')
send = gpd.read_file(send_data).to_crs('EPSG:4269')
hdist = gpd.read_file(hdist_data).to_crs('EPSG:4269')

In [44]:
cong_df.head()

,LONGNAME,SHORTNAME,DISTRICT,COLOR,TARGET_DEV,TOTAL,TARGET,DEVIATION_,DEVIATION,geometry
0,Congressional 1,CD1,1,434209920,-1241,726395,727636,-0.17,-1241,"POLYGON ((-157.76009 21.21439, -157.76096 21.2..."
1,Congressional 2,CD2,2,-124684160,1240,728876,727636,0.17,1240,"MULTIPOLYGON (((-159.61920 22.26684, -159.6129..."


In [45]:
send.head()

,LONGNAME,SHORTNAME,DISTRICT,COLOR,TOTAL,TARGET,TARGET_DEV,PopHaw2022,geometry
0,Senate 1,S1,1,-5800064,50485,49998,0.04,50019,"POLYGON ((-155.03001 19.79211, -155.02992 19.7..."
1,Senate 2,S2,2,-390135936,50344,49998,0.57,50284,"POLYGON ((-155.25790 19.54401, -155.25001 19.5..."
2,Senate 3,S3,3,-142612876,49943,49998,-0.22,49887,"POLYGON ((-155.34410 19.52961, -155.34398 19.5..."
3,Senate 4,S4,4,1944059776,49857,49998,-0.40,49800,"POLYGON ((-155.12296 19.90515, -155.12035 19.9..."
4,Senate 8,S8,8,-724345472,73298,73004,0.00,73004,"MULTIPOLYGON (((-160.30437 21.85380, -160.3023..."


In [46]:
hdist.head()

,LONGNAME,SHORTNAME,DISTRICT,COLOR,TOTAL,TARGET_DEV,TARGET,PopHaw2022,geometry
0,House 1,H1,1,873226624,25246,0.67,24999,25166,"POLYGON ((-155.08910 19.72524, -155.08916 19.7..."
1,House 2,H2,2,-602205056,25215,-0.36,24999,24908,"POLYGON ((-155.14257 19.65955, -155.13672 19.6..."
2,House 3,H3,3,1957388416,25170,0.14,24999,25035,"POLYGON ((-155.08525 19.74166, -155.08524 19.7..."
3,House 4,H4,4,-858171264,25080,0.26,24999,25065,"POLYGON ((-154.93027 19.59374, -154.93023 19.5..."
4,House 5,H5,5,2048902272,25515,1.94,24999,25485,"POLYGON ((-155.51166 19.66243, -155.50988 19.6..."


In [47]:
election_df = add_district(cong_df, "CD", election_df, "DISTRICT")

100%|███████████████████████████████████████████████████| 2/2 [00:00<00:00, 16.73it/s]


In [48]:
election_df = add_district(send, "SEND", election_df, "DISTRICT")

100%|█████████████████████████████████████████████████| 25/25 [00:00<00:00, 53.23it/s]


In [49]:
election_df = add_district(hdist, "HDIST", election_df, "DISTRICT")

100%|█████████████████████████████████████████████████| 51/51 [00:00<00:00, 54.27it/s]


In [50]:
election_df.columns

Index(['2MOREVAP', 'AMINVAP', 'ASIANVAP', 'BVAP', 'COUNTYFP', 'GEOID',
       'GOV18D', 'GOV18O', 'GOV18R', 'HISP', 'HVAP', 'H_2MORE', 'H_AMIN',
       'H_ASIAN', 'H_BLACK', 'H_NHPI', 'H_OTHER', 'H_WHITE', 'NHPIVAP',
       'NH_2MORE', 'NH_AMIN', 'NH_ASIAN', 'NH_BLACK', 'NH_NHPI', 'NH_OTHER',
       'NH_WHITE', 'OTHERVAP', 'PRE16D', 'PRE16O', 'PRE16R', 'PRE20D',
       'PRE20O', 'PRE20R', 'PRECINCT', 'STATEFP', 'TOTPOP', 'USS16D', 'USS16O',
       'USS16R', 'USS18D', 'USS18R', 'VAP', 'WVAP', 'geometry', 'CD', 'SEND',
       'HDIST'],
      dtype='object')

### Put the base precinct year after the precinct information column

In [60]:
base_columns = {}
if 'COUNTYFP' + year not in election_df.columns:
    base_columns = {
        'STATEFP':'STATEFP'+year,
        'COUNTYFP':'COUNTYFP'+year,
        'PRECINCT':'PRECINCT'+year,
        'GEOID':'GEOID'+year,
        'NAME':'NAME'+year}
election_df.rename(columns=base_columns, inplace = True)

In [64]:
# reorder the columns
fixed_columns = [
    'STATEFP'+year,
    'COUNTYFP'+year,
    'PRECINCT'+year,
    'GEOID'+year,
    # 'NAME'+year,
    'CD',
    'SEND',
    'HDIST',
    'TOTPOP',
    'NH_2MORE',
    'NH_AMIN',
    'NH_ASIAN',
    'NH_BLACK',
    'NH_NHPI',
    'NH_OTHER',
    'NH_WHITE',
    'HISP',
    'H_AMIN',
    'H_ASIAN',
    'H_BLACK',
    'H_NHPI',
    'H_OTHER',
    'H_WHITE',
    'H_2MORE',
    'VAP',
    'HVAP',
    'WVAP',
    'BVAP',
    'AMINVAP',
    'ASIANVAP',
    'NHPIVAP',
    'OTHERVAP',
    '2MOREVAP']

election_columns = [col for col in election_df.columns if col not in fixed_columns]
final_col = fixed_columns + election_columns
election_df = election_df[final_col]

In [65]:
list(election_df.columns)

['STATEFP20',
 'COUNTYFP20',
 'PRECINCT20',
 'GEOID20',
 'CD',
 'SEND',
 'HDIST',
 'TOTPOP',
 'NH_2MORE',
 'NH_AMIN',
 'NH_ASIAN',
 'NH_BLACK',
 'NH_NHPI',
 'NH_OTHER',
 'NH_WHITE',
 'HISP',
 'H_AMIN',
 'H_ASIAN',
 'H_BLACK',
 'H_NHPI',
 'H_OTHER',
 'H_WHITE',
 'H_2MORE',
 'VAP',
 'HVAP',
 'WVAP',
 'BVAP',
 'AMINVAP',
 'ASIANVAP',
 'NHPIVAP',
 'OTHERVAP',
 '2MOREVAP',
 'GOV18D',
 'GOV18O',
 'GOV18R',
 'PRE16D',
 'PRE16O',
 'PRE16R',
 'PRE20D',
 'PRE20O',
 'PRE20R',
 'USS16D',
 'USS16O',
 'USS16R',
 'USS18D',
 'USS18R',
 'geometry']

In [66]:
# store the result in directory "il"
os.makedirs("./{}".format(state_ab))
election_df.to_file("./{}/{}.shp".format(state_ab, state_ab))
election_df.to_file('./{}/{}.geojson'.format(state_ab, state_ab), driver='GeoJSON')

# Only do once to build json and read from file when generating ensembles
graph = Graph.from_file("./{}/{}.shp".format(state_ab, state_ab), ignore_errors=True)
graph.to_json("./{}/{}.json".format(state_ab, state_ab))